# German Traffic Sign Classifier
### By Luke Oltmanns & Ryan Peerenboom

# Loading The Data

In [ ]:
from __future__ import print_function

import pandas as pd
import numpy as np
import os

import csv
import cv2

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import datasets, layers, models

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from skimage import data as skimage_data
from skimage import transform
from skimage.color import rgb2gray

In [ ]:
#paths to data
root = "./drive/MyDrive/Dataset/"
trainPath = "./drive/MyDrive/Dataset/Train.csv"
testPath = "./drive/MyDrive/Dataset/Test.csv"

In [ ]:
columns = ["Width", "Height", "Roi.X1", "Roi.Y1", "Roi.X2", "Roi.Y2", "ClassId", "Path"]

In [ ]:
#method for loading training data
def loadDataset(path):
    images_histogram_equalized = []
    images = []
    classes = []
    data = pd.read_csv(path)
    data = data.sample(frac=1).reset_index(drop=True)

    # loop to pull images from csv one row at a time
    for i,row in data.iterrows():
        img_path = row['Path']
        img_class = row['ClassId']

        img = os.path.join(root,img_path)
        img = cv2.imread(img)

        # bypasses any image not loaded to avoid erros
        if type(img) is np.ndarray:
            if img.size == 0:
                continue
        if img is None:
            continue

        # resize image to disired size
        img_resize = cv2.resize(img,(32,32),3) #resizes each img to size 32*32*3

        #split and perform histogram equalization to enhance image contrast
        R, G, B = cv2.split((img_resize))

        img_r = cv2.equalizeHist(R)
        img_g = cv2.equalizeHist(G)
        img_b = cv2.equalizeHist(B)

        new_image = cv2.merge((img_r, img_g, img_b))

        # put data into numpy arrays
        if i>0 and i%1000 == 0:
            print("loaded: ",i," images")
        images.append(img_resize)
        images_histogram_equalized.append(new_image)
        classes.append(img_class)

    # put data into numpy arrays
    imgs = np.array(images)
    imgs_hist = np.array(images_histogram_equalized)
    labels = np.array(classes)
    
    return imgs, imgs_hist, labels

In [ ]:
#use crated 'loadDataset()' methods to retrieve training images and labels
X_train, y_train = loadDataset(trainPath)

In [ ]:
#use crated 'loadDataset()' methods to retrieve testing images and labels
X_test, y_test = loadDataset(testPath)

In [ ]:
print("X_train: ", X_train.shape)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test: ", y_test.shape)

In [ ]:
#Normalize pixel values between 0 and 1
X_train, X_train_hist, X_test, X_test_hist= X_train/255.0, X_train_hist/255.0, X_test/255.0, X_test_hist/255.0

### Displaying the Data

In [ ]:
#verify dataset
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])

plt.show()

In [ ]:
#verify training data as a bar graph
n_classes = max(y_test) - min(y_test) + 1
classes = range(n_classes)
counts = []

for sign_code in range(n_classes):
    image_list = list(np.where(y_train == sign_code)[0])
    counts += [len(image_list)]
    
plt.figure(figsize=(15,5)) 
plt.bar(classes, counts, width=0.7, align='center')
plt.ylabel('Number of Examples')
plt.xlabel('Class Index')
plt.xticks(np.arange(0, 43, 1.0))
plt.title("Class Distribution of Training Set")
plt.ylim([0,4000])
plt.show()

In [ ]:
#verify testing data as a bar graph
n_classes = max(y_test) - min(y_test) + 1
classes = range(n_classes)
counts = []

for sign_code in range(n_classes):
    image_list = list(np.where(y_test == sign_code)[0])
    counts += [len(image_list)]
    
plt.figure(figsize=(15,5)) 
plt.bar(classes, counts, width=0.7, align='center')
plt.ylabel('Number of Examples')
plt.xlabel('Class Index')
plt.xticks(np.arange(0, 43, 1.0))
plt.title("Class Distribution of Testing Set")
plt.ylim([0,4000])
plt.show()

# Building Models

### CNN ReLU

In [ ]:
#create convolutional Neural Network (CNN) with base "relu"
CNNModelRelu = models.Sequential()
CNNModelRelu.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
CNNModelRelu.add(layers.MaxPooling2D((2, 2)))
CNNModelRelu.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNModelRelu.add(layers.MaxPooling2D((2, 2)))
CNNModelRelu.add(layers.Conv2D(64, (3, 3), activation='relu'))
CNNModelRelu.add(layers.Dropout(0.2))

#add dense layers
CNNModelRelu.add(layers.Flatten())
CNNModelRelu.add(layers.Dense(64, activation='relu'))
CNNModelRelu.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelRelu
CNNModelRelu.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryRelu = CNNModelRelu.fit(X_train_hist, y_train, epochs=10, validation_data=(X_test_hist, y_test))

### CNN Tanh

In [ ]:
#create convolutional Neural Network (CNN) with base "tanh"
CNNModelTanh= models.Sequential()
CNNModelTanh.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
CNNModelTanh.add(layers.MaxPooling2D((2, 2)))
CNNModelTanh.add(layers.Conv2D(64, (3, 3), activation='tanh'))
CNNModelTanh.add(layers.MaxPooling2D((2, 2)))
CNNModelTanh.add(layers.Conv2D(64, (3, 3), activation='tanh'))
CNNModelTanh.add(layers.Dropout(0.2))

#add dense layers
CNNModelTanh.add(layers.Flatten())
CNNModelTanh.add(layers.Dense(64, activation='tanh'))
CNNModelTanh.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelTanh
CNNModelTanh.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryTanh= CNNModelTanh.fit(X_train_hist, y_train, epochs=10, validation_data=(X_test_hist, y_test))

### CNN Sigmoid

In [ ]:
#create convolutional Neural Network (CNN) with base "sigmoid"
CNNModelSig= models.Sequential()
CNNModelSig.add(layers.Conv2D(32, (3, 3), activation='sigmoid', input_shape=(32, 32, 3)))
CNNModelSig.add(layers.MaxPooling2D((2, 2)))
CNNModelSig.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
CNNModelSig.add(layers.MaxPooling2D((2, 2)))
CNNModelSig.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
CNNModelSig.add(layers.Dropout(0.2))

#add dense layers
CNNModelSig.add(layers.Flatten())
CNNModelSig.add(layers.Dense(64, activation='sigmoid'))
CNNModelSig.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelSig
CNNModelSig.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistorySig = CNNModelSig.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### CNN Cross Activations

In [ ]:
#create convolutional Neural Network (CNN) with cross activations
CNNModelX= models.Sequential()
CNNModelX.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 3)))
CNNModelX.add(layers.MaxPooling2D((2, 2)))
CNNModelX.add(layers.Conv2D(64, (3, 3), activation='tanh'))
CNNModelX.add(layers.MaxPooling2D((2, 2)))
CNNModelX.add(layers.Conv2D(64, (3, 3), activation='sigmoid'))
CNNModelX.add(layers.Dropout(0.1))

#add dense layers
CNNModelX.add(layers.Flatten())
CNNModelX.add(layers.Dense(64, activation='relu'))
CNNModelX.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelX
CNNModelX.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryX= CNNModelX.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### CNN Random Layers

In [ ]:
#create random convolutional Neural Network (CNN)
CNNModelRand= models.Sequential()
CNNModelRand.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
CNNModelRand.add(layers.MaxPooling2D((2, 2)))
CNNModelRand.add(layers.Conv2D(48, (5, 5), activation='relu'))
CNNModelRand.add(layers.MaxPooling2D((2, 2)))
CNNModelRand.add(layers.Dropout(0.2))

#add dense layers
CNNModelRand.add(layers.Flatten())
CNNModelRand.add(layers.Dense(128, activation='relu'))
CNNModelRand.add(layers.Dense(64, activation='relu'))
CNNModelRand.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelRand
CNNModelRand.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryRand= CNNModelRand.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### CNN Random Layers Two

In [ ]:
#create second random convolutional Neural Network (CNN)
CNNModelRandTwo= models.Sequential()
CNNModelRandTwo.add(layers.Conv2D(32, (3, 3), activation='tanh', input_shape=(32, 32, 3)))
CNNModelRandTwo.add(layers.MaxPooling2D((2, 2)))
CNNModelRandTwo.add(layers.Conv2D(128, (2, 2), activation='relu'))
CNNModelRandTwo.add(layers.Conv2D(64, (5, 5), activation='relu'))
CNNModelRandTwo.add(layers.MaxPooling2D((3, 3)))
CNNModelRandTwo.add(layers.Dropout(0.4))

#add dense layers
CNNModelRandTwo.add(layers.Flatten())
CNNModelRandTwo.add(layers.Dense(512, activation='relu'))
CNNModelRandTwo.add(layers.Dense(256, activation='tanh'))
CNNModelRandTwo.add(layers.Dropout(0.2))
CNNModelRandTwo.add(layers.Dense(512, activation='relu'))
CNNModelRandTwo.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelRandTwo
CNNModelRandTwo.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryRandTwo= CNNModelRandTwo.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### Simple Dense NN 0 hidden Layers

In [ ]:
#simple dense neural network with 0 hidden layers of ReLU
SDNNModelZero = models.Sequential()
SDNNModelZero.add(layers.Flatten())
SDNNModelZero.add(layers.Dense(43, activation='softmax'))
SDNNModelZero.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

SDNNHistoryZero= SDNNModelZero.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### Simple Dense NN 5 hidden Layers

In [ ]:
#simple dense neural network with 5 hidden layers of ReLU
SDNNModelFiveRelu = models.Sequential()
SDNNModelFiveRelu.add(layers.Flatten())
SDNNModelFiveRelu.add(layers.Dense(1024, activation='relu'))
SDNNModelFiveRelu.add(layers.Dense(512, activation='relu'))
SDNNModelFiveRelu.add(layers.Dense(256, activation='relu'))
SDNNModelFiveRelu.add(layers.Dense(128, activation='relu'))
SDNNModelFiveRelu.add(layers.Dense(64, activation='relu'))
SDNNModelFiveRelu.add(layers.Dense(43, activation='softmax'))
SDNNModelFiveRelu.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

SDNNHistoryFiveRelu= SDNNModelFiveRelu.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### CNN Best Result

In [ ]:
#create convolutional neural network with best result
CNNModelBest= models.Sequential()
CNNModelBest.add(layers.Conv2D(32, (5, 5), activation='relu', input_shape=(32, 32, 3)))
CNNModelBest.add(layers.BatchNormalization())
CNNModelBest.add(layers.MaxPooling2D((2, 2)))

CNNModelBest.add(layers.Conv2D(48, (5, 5), activation='relu'))
CNNModelBest.add(layers.BatchNormalization())
CNNModelBest.add(layers.MaxPooling2D((2, 2)))

#add dense layers
CNNModelBest.add(layers.Flatten())
CNNModelBest.add(layers.Dense(120, activation='relu'))
CNNModelBest.add(layers.Dropout(0.2))
CNNModelBest.add(layers.Dense(84, activation='relu'))
CNNModelBest.add(layers.Dropout(0.1))
CNNModelBest.add(layers.Dense(43, activation='softmax'))

#compile and train the CNNModelBest
CNNModelBest.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

CNNHistoryBest= CNNModelBest.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Testing the Models

### Standard CNN Models

In [ ]:
#display traditional CNN models' accuracies
plt.plot(CNNHistoryRelu.history['acc'], label='CNN ReLU accuracy')
plt.plot(CNNHistoryRelu.history['val_acc'], label='CNN ReLU val_accuracy')

plt.plot(CNNHistoryTanh.history['acc'], label='CNN Tanh accuracy')
plt.plot(CNNHistoryTanh.history['val_acc'], label='CNN Tanh val_accuracy')

plt.plot(CNNHistorySig.history['acc'], label='CNN Sigmoid accuracy')
plt.plot(CNNHistorySig.history['val_acc'], label='CNN Sigmoid val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
#display traditional CNN models' loss
plt.plot(CNNHistoryRelu.history['loss'], label='CNN ReLU loss')
plt.plot(CNNHistoryRelu.history['val_loss'], label='CNN ReLU val_loss')

plt.plot(CNNHistoryTanh.history['loss'], label='CNN Tanh loss')
plt.plot(CNNHistoryTanh.history['val_loss'], label='CNN Tanh val_loss')

plt.plot(CNNHistorySig.history['loss'], label='CNN Sigmoid loss')
plt.plot(CNNHistorySig.history['val_loss'], label='CNN Sigmoid val_loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.5])
plt.legend(loc='lower right')

In [ ]:
#evaluate traditional CNN models
relu_test_loss, relu_test_acc = CNNModelRelu.evaluate(X_test, y_test, verbose = 2)
print("The ReLU test accuracy is: " + str(relu_test_acc))
print("The ReLU test loss is: " + str(relu_test_loss))

tanh_test_loss, tanh_test_acc = CNNModelTanh.evaluate(X_test, y_test, verbose = 2)
print("The Tanh test accuracy is: " + str(tanh_test_acc))
print("The Tanh test loss is: " + str(tanh_test_loss))

sig_test_loss, sig_test_acc = CNNModelSig.evaluate(X_test, y_test, verbose = 2)
print("The Sigmoid test accuracy is: " + str(sig_test_acc))
print("The Sigmoid test loss is: " + str(sig_test_loss))

### Exotic CNN Models

In [ ]:
#display exotic CNN models' accuracies
plt.plot(CNNHistoryX.history['acc'], label='CNN Cross accuracy')
plt.plot(CNNHistoryX.history['val_acc'], label='CNN Cross val_accuracy')

plt.plot(CNNHistoryRand.history['acc'], label='CNN Random accuracy')
plt.plot(CNNHistoryRand.history['val_acc'], label='CNN Random val_accuracy')

plt.plot(CNNHistoryRandTwo.history['acc'], label='CNN Random Two accuracy')
plt.plot(CNNHistoryRandTwo.history['val_acc'], label='CNN Random Two val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
#evaluate exotic CNN models
X_test_loss, X_test_acc = CNNModelX.evaluate(X_test, y_test, verbose = 2)
print("The Cross test accuracy is: " + str(X_test_acc))

Rand_test_loss, Rand_test_acc = CNNModelRand.evaluate(X_test, y_test, verbose = 2)
print("The Random test accuracy is: " + str(Rand_test_acc))

RandTwo_test_loss, RandTwo_test_acc = CNNModelRandTwo.evaluate(X_test, y_test, verbose = 2)
print("The Second Random test accuracy is: " + str(sig_test_acc))

### Simple Dense NN Models

In [ ]:
#display SDNN models' accuracies
plt.plot(SDNNHistoryZero.history['acc'], label='SDNN Zero accuracy')
plt.plot(SDNNHistoryZero.history['val_acc'], label='SDNN Zero val_accuracy')

plt.plot(SDNNHistoryFiveRelu.history['acc'], label='SDNN Five accuracy')
plt.plot(SDNNHistoryFiveRelu.history['val_acc'], label='SDNN Five val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
#evaluate exotic CNN models
Zero_test_loss, Zero_test_acc = SDNNModelZero.evaluate(X_test, y_test, verbose = 2)
print("The zero hidden layer test accuracy is: " + str(Zero_test_acc))

Five_test_loss, Rand_test_acc = SDNNModelFiveRelu.evaluate(X_test, y_test, verbose = 2)
print("The five hidden layer test accuracy is: " + str(Rand_test_acc))

### Best Result Model

In [ ]:
#display 'Best' model accuracies
plt.plot(CNNHistoryBest.history['acc'], label='CNN Best Result accuracy')
plt.plot(CNNHistoryBest.history['val_acc'], label='CNN Best Result val_accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
#display 'Best' model loss
plt.plot(CNNHistoryBest.history['loss'], label='CNN Best Result loss')
plt.plot(CNNHistoryBest.history['val_loss'], label='CNN Best Result val_loss')

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 0.5])
plt.legend(loc='lower right')

In [ ]:
#evaluate 'Best' model
best_test_loss, best_test_acc = CNNModelBest.evaluate(X_test, y_test, verbose = 2)
print("The Best Result test accuracy is: " + str(best_test_acc))
print("The Best Result test loss is: " + str(best_test_loss))

### All Model Accuracies

In [ ]:
#display all models' accuracies
plt.plot(CNNHistoryRelu.history['acc'], label='CNN ReLU accuracy')
plt.plot(CNNHistoryTanh.history['acc'], label='CNN Tanh accuracy')
plt.plot(CNNHistorySig.history['acc'], label='CNN Sigmoid accuracy')
plt.plot(CNNHistoryX.history['acc'], label='CNN Cross accuracy')
plt.plot(CNNHistoryRand.history['acc'], label='CNN Random accuracy')
plt.plot(CNNHistoryRandTwo.history['acc'], label='CNN Random Two accuracy')
plt.plot(SDNNHistoryZero.history['acc'], label='SDNN Zero accuracy')
plt.plot(SDNNHistoryFiveRelu.history['acc'], label='SDNN Five accuracy')
plt.plot(CNNHistoryBest.history['acc'], label='CNN Best Result accuracy')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

## Make Predictions

In [ ]:
#verify that our model is accurate by showing that an image from 
#our test data matches a predicted image from our model evaluation
probability_model = tf.keras.Sequential([CNNModelBest, tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)
predictions[0]
np.argmax(predictions[0])
y_test[0]

## Verify Predictions

In [ ]:
#array of labels (corresponds with the bar graphs dispayed above)
class_names = ['0','1','2','3','4','5','6','7','8','9',
               '10','11','12','13','14','15','16','17','18','19',
               '20','21','22','23','24','25','26','27','28','29',
               '30','31','32','33','34','35','36','37','38','39',
               '40','41','42']

In [ ]:
#method for displaying images
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)


In [ ]:
#method for displaying graph of image classification accuracy
def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(43))
  plt.yticks([])
  thisplot = plt.bar(range(43), predictions_array, color="#777777")
  plt.ylim([0, 0.1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
#display a single image and visual result
i = 0
plt.figure(figsize=(30,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_test, X_test)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], y_test)
plt.show()

In [ ]:
#display a single image and visual result
i = 12
plt.figure(figsize=(30,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], y_test, X_test)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i], y_test)
plt.show()

In [ ]:
#display a multiple images and visual result of those images
num_rows = 15
num_cols = 1
num_images = num_rows*num_cols
plt.figure(figsize=(2*20*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], y_test, X_test)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], y_test)
plt.tight_layout()
plt.show()